In [1]:
pip install --upgrade mne

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# %%
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import mne

import models

# 데이터 로드 함수 (Physionet_DataLoad 모듈 없이 직접 사용)
def load_physionet(data_path):
    exclude = [38, 88, 89, 92, 100, 104]  # 제외할 피험자 번호
    subjects = [n for n in np.arange(1, 110) if n not in exclude]

    xs = list()
    ys = list()
    for subject in subjects:
        x, y, ch_names = load_subject_data(subject, data_path, True)
        xs.append(x)
        ys.append(y)
    data_x = np.concatenate(xs)
    data_y = np.concatenate(ys)

    N_tr, N_ch, _ = data_x.shape
    data_x = data_x[:, :, :640].reshape(N_tr, 1, N_ch, -1)
    y_one_hot = to_one_hot(data_y, by_sub=False)

    # Create Validation/test
    x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(
        data_x,
        y_one_hot,
        stratify=y_one_hot,
        test_size=0.10,
        random_state=42
    )

    # Scale independently train/test
    x_train_scaled_raw, x_test_valid_scaled_raw = standardize_data(x_train_raw, x_valid_test_raw, N_ch)

    print(x_train_scaled_raw.shape, x_test_valid_scaled_raw.shape)
    return x_train_scaled_raw, y_train_raw, x_test_valid_scaled_raw, y_valid_test_raw



# 나머지 코드는 동일하게 유지
# 기존의 train, test 및 모델 정의 함수들은 그대로 둡니다.


2024-08-22 12:43:18.248441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 12:43:18.248535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 12:43:18.248564: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 12:43:18.257628: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packa

In [5]:
def standardize_data(X_train, X_test, channels):
    for j in range(channels):
        scaler = StandardScaler()
        scaler.fit(X_train[:, 0, j, :])
        X_train[:, 0, j, :] = scaler.transform(X_train[:, 0, j, :])
        X_test[:, 0, j, :] = scaler.transform(X_test[:, 0, j, :])
    return X_train, X_test


In [6]:
# One-hot 인코딩 함수
def to_one_hot(y, by_sub=False):
    if by_sub:
        new_array = np.array(["nan" for nan in range(len(y))])
        for index, label in enumerate(y):
            new_array[index] = ''.join([i for i in label if not i.isdigit()])
    else:
        new_array = y.copy()
    total_labels = np.unique(new_array)
    mapping = {label: idx for idx, label in enumerate(total_labels)}
    for x in range(len(new_array)):
        new_array[x] = mapping[new_array[x]]

    return keras.utils.to_categorical(new_array)

In [7]:

# 데이터 로드
def load_subject_data(subject: int, data_path: str, exclude_base: bool = False):
    runs = [4, 6, 8, 10, 12, 14]
    task2 = [4, 8, 12]
    task4 = [6, 10, 14]
    sub_name = f"S{subject:03d}"
    sub_folder = os.path.join(data_path, sub_name)
    subject_runs = []

    for run in runs:
        path_run = os.path.join(sub_folder, f"{sub_name}R{run:02d}.edf")
        raw_run = mne.io.read_raw_edf(path_run, preload=True)
        len_run = np.sum(raw_run._annotations.duration)
        if len_run > 124:
            raw_run.crop(tmax=124)

        if run in task2:
            for index, an in enumerate(raw_run.annotations.description):
                if an == "T0":
                    raw_run.annotations.description[index] = "B"
                if an == "T1":
                    raw_run.annotations.description[index] = "L"
                if an == "T2":
                    raw_run.annotations.description[index] = "R"
        if run in task4:
            for index, an in enumerate(raw_run.annotations.description):
                if an == "T0":
                    raw_run.annotations.description[index] = "B"
                if an == "T1":
                    raw_run.annotations.description[index] = "LR"
                if an == "T2":
                    raw_run.annotations.description[index] = "F"
        subject_runs.append(raw_run)

    raw_conc = mne.concatenate_raws(subject_runs)
    mne.datasets.eegbci.standardize(raw_conc)  # eegbci 모듈의 위치를 명시적으로 지정
    montage = mne.channels.make_standard_montage('standard_1005')
    raw_conc.set_montage(montage)

    tmin = 0
    tmax = 4
    event_id = dict(F=2, L=3, LR=4, R=5) if exclude_base else dict(B=1, F=2, L=3, LR=4, R=5)
    events, _ = mne.events_from_annotations(raw_conc, event_id=event_id)

    picks = mne.pick_types(raw_conc.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

    epochs = mne.Epochs(raw_conc, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    y = [epochs[index]._name for index in range(len(epochs))]
    xs = np.array([epoch for epoch in epochs])

    return xs, y, raw_conc.ch_names


In [8]:
def prepare_dataset(data_path):
    """
    Physionet 데이터셋을 로드하고 전처리하는 함수.
    
    Parameters:
    - data_path (str): 데이터셋이 저장된 경로.

    Returns:
    - X_train, y_train: 훈련 데이터 및 레이블.
    - X_test, y_test: 테스트 데이터 및 레이블.
    """
    X_train, y_train, X_test, y_test = load_physionet(data_path)
    return X_train, y_train, X_test, y_test

In [9]:
def train_model(model, dataset_conf, train_conf, results_path, X_train, y_train, X_test, y_test):
    """
    모델을 훈련하는 함수.
    
    Parameters:
    - model (keras.Model): 훈련할 모델.
    - dataset_conf (dict): 데이터셋 관련 설정.
    - train_conf (dict): 훈련 관련 설정.
    - results_path (str): 훈련 결과를 저장할 경로.
    - X_train, y_train: 훈련 데이터 및 레이블.
    - X_test, y_test: 테스트 데이터 및 레이블.
    """
    # 모델 요약 출력
    model.summary()

    # 모델 훈련 (train 함수가 이미 구현되어 있다고 가정)
    train(dataset_conf, train_conf, results_path, X_train, y_train, X_test, y_test)

In [9]:
def run():
    """
    데이터셋 경로를 설정하고 모델을 생성하여 훈련하는 메인 함수.
    """
    # 데이터셋 경로 설정
    data_path = "/home/work/HCOH/motors/eeg_motor_files"

    # 데이터셋 준비
    X_train, y_train, X_test, y_test = prepare_dataset(data_path)

    # 결과 저장 경로 생성
    results_path = os.getcwd() + "/results"
    if not os.path.exists(results_path):
        os.makedirs(results_path)

    # 데이터셋 파라미터 설정
    dataset_conf = {'n_classes': 4, 'n_channels': 64, 'data_path': data_path}
    
    # 훈련 하이퍼파라미터 설정
    train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009,
                  'LearnCurves': True, 'model': 'DB_ATCNet'}

    # 모델 생성
    model = getModel(train_conf.get('model'))
    if model is None:
        print("Error: The model could not be created.")
        return

    # 모델 훈련
    train_model(model, dataset_conf, train_conf, results_path, X_train, y_train, X_test, y_test)

In [10]:
def run():
    # 데이터셋 경로 설정
    data_path = "/home/work/HCOH/motors/eeg_motor_files"

    # 데이터 로드
    X_train, y_train, X_test, y_test = load_physionet(data_path)

    # 결과 저장 경로 생성
    results_path = os.getcwd() + "/results"
    if not os.path.exists(results_path):
        os.makedirs(results_path)

    # 데이터셋 파라미터 설정
    dataset_conf = {'n_classes': 4, 'n_channels': 64, 'data_path': data_path}
    
    # 훈련 하이퍼파라미터 설정
    train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009,
                  'LearnCurves': True, 'model': 'simple_cnn'}

    model = getModel(train_conf.get('model'))
    if model is not None:
        model.summary()
    else:
        print("Error: The model could not be created.")

    # 모델 훈련
    train(dataset_conf, train_conf, results_path, X_train, y_train, X_test, y_test)

In [11]:
def train(dataset_conf, train_conf, results_path):
    in_exp = time.time()

    # 파일 및 경로 설정
    best_model_path = os.path.join(results_path, "best_model.h5")
    log_file_path = os.path.join(results_path, "log.txt")

    # 데이터셋 및 모델 설정
    data_path = dataset_conf.get('data_path')
    X_train, y_train_onehot, X_test, y_test_onehot = load_physionet(data_path)

    model_name = train_conf.get('model')
    model = getModel(model_name)
    model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=train_conf.get('lr')), metrics=['accuracy'])

    callbacks = [
        ModelCheckpoint(best_model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
        EarlyStopping(monitor='val_accuracy', verbose=1, mode='max', patience=train_conf.get('patience'))
    ]

    history = model.fit(X_train, y_train_onehot, validation_data=(X_test, y_test_onehot),
                        epochs=train_conf.get('epochs'), batch_size=train_conf.get('batch_size'),
                        callbacks=callbacks, verbose=1)

    model.load_weights(best_model_path)
    y_pred = model.predict(X_test).argmax(axis=-1)
    labels = y_test_onehot.argmax(axis=-1)
    acc = accuracy_score(labels, y_pred)
    kappa = cohen_kappa_score(labels, y_pred)

    with open(log_file_path, "w") as log_write:
        info = f'Subject: 1   Test_acc: {acc:.5f}   Test_kappa: {kappa:.5f}'
        print(info)
        log_write.write(info + '\n')

    if train_conf.get('LearnCurves'):
        draw_learning_curves(history, results_path)

    out_exp = time.time()
    with open(log_file_path, "a") as log_write:
        log_write.write(f'Total Training Time: {(out_exp - in_exp) / 3600:.3f} hours\n')

    test(model, dataset_conf, results_path, X_test, y_test_onehot)


# %% Evaluation
def test(model, dataset_conf, results_path, X_test, y_test_onehot, allRuns=True):
    log_file_path = os.path.join(results_path, "log.txt")
    n_classes = dataset_conf.get('n_classes')
    cf_matrix = np.zeros([n_classes, n_classes])

    y_pred = model.predict(X_test).argmax(axis=-1)
    labels = y_test_onehot.argmax(axis=-1)
    acc_bestRun = accuracy_score(labels, y_pred)
    kappa_bestRun = cohen_kappa_score(labels, y_pred)
    cf_matrix[:, :] = confusion_matrix(labels, y_pred, normalize='pred')
    draw_confusion_matrix(cf_matrix, results_path)

    with open(log_file_path, "a") as log_write:
        info = f'Subject: 1   acc: {acc_bestRun:.5f}   kappa: {kappa_bestRun:.5f}\n'
        print(info)
        log_write.write(info)

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.constraints import max_norm

def getModel(model_name='simple_cnn', input_shape=(1, 64, 640), n_classes=4, **kwargs):
    """
    Returns a simple CNN model.

    Parameters:
    - model_name (str): Name of the model to create (default is 'simple_cnn').
    - input_shape (tuple): Shape of the input data (default is (28, 28, 1)).
    - n_classes (int): Number of classes for the output layer (default is 10).

    Returns:
    - model: A Keras Model instance.
    """
    if model_name == 'simple_cnn':
        inputs = Input(shape=input_shape)

        # First convolutional block
        x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
        x = MaxPooling2D(pool_size=(1, 2))(x)  # Adjusted pooling size to (1, 2)
        x = Dropout(0.25)(x)

        # Second convolutional block
        x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)  # Adjusted pooling size to (1, 2)
        x = Dropout(0.25)(x)

        # Flatten and fully connected layer
        x = Flatten()(x)
        x = Dense(128, activation='relu', kernel_constraint=max_norm(3.))(x)
        x = Dropout(0.5)(x)

        # Output layer
        outputs = Dense(n_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model
    else:
        raise ValueError(f"Model '{model_name}' is not recognized.")

In [15]:
if __name__ == "__main__":
    run()

Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S001/S001R12.e

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S006/S006R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S006/S006R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S006/S006R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S006/S006R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
R

Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S011/S011R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S011/S011R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S011/S011R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S011/S011R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs.

Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S016/S016R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S016/S016R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S016/S016R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S016/S016R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/work/HCOH/motors/eeg_motor_files/S016/S016R14.e

KeyboardInterrupt: 

In [23]:
import os
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_or_create_dataset(data_path, save_path):
    # 데이터셋이 이미 저장되어 있는지 확인
    if os.path.exists(save_path):
        print("Loading dataset from disk...")
        npzfile = np.load(save_path)
        return npzfile['x_train'], npzfile['y_train'], npzfile['x_test'], npzfile['y_test']

    print("Loading dataset from source...")
    exclude = [38, 88, 89, 92, 100, 104]
    subjects = [n for n in np.arange(1, 110) if n not in exclude]

    xs = list()
    ys = list()
    for subject in subjects:
        x, y, ch_names = load_subject_data(subject, data_path, True)
        xs.append(x)
        ys.append(y)
    data_x = np.concatenate(xs)
    data_y = np.concatenate(ys)

    N_tr, N_ch, _ = data_x.shape
    data_x = data_x[:, :, :640].reshape(N_tr, 1, N_ch, -1)
    y_one_hot = to_one_hot(data_y, by_sub=False)

    # Create Validation/test
    x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(
        data_x,
        y_one_hot,
        stratify=y_one_hot,
        test_size=0.10,
        random_state=42
    )

    # Scale independently train/test
    x_train_scaled_raw, x_test_valid_scaled_raw = standardize_data(x_train_raw, x_valid_test_raw, N_ch)

    # 데이터셋을 디스크에 저장
    np.savez(save_path, x_train=x_train_scaled_raw, y_train=y_train_raw, x_test=x_test_valid_scaled_raw, y_test=y_valid_test_raw)

    print(x_train_scaled_raw.shape, x_test_valid_scaled_raw.shape)
    return x_train_scaled_raw, y_train_raw, x_test_valid_scaled_raw, y_valid_test_raw

def load_subject_data(subject: int, data_path: str, exclude_base: bool = False):
    runs = [4, 6, 8, 10, 12, 14]
    task2 = [4, 8, 12]
    task4 = [6, 10, 14]
    sub_name = f"S{subject:03d}"
    sub_folder = os.path.join(data_path, sub_name)
    subject_runs = []

    for run in runs:
        path_run = os.path.join(sub_folder, f"{sub_name}R{run:02d}.edf")
        raw_run = mne.io.read_raw_edf(path_run, preload=True)
        len_run = np.sum(raw_run._annotations.duration)
        if len_run > 124:
            raw_run.crop(tmax=124)

        if run in task2:
            for index, an in enumerate(raw_run.annotations.description):
                if an == "T0":
                    raw_run.annotations.description[index] = "B"
                if an == "T1":
                    raw_run.annotations.description[index] = "L"
                if an == "T2":
                    raw_run.annotations.description[index] = "R"
        if run in task4:
            for index, an in enumerate(raw_run.annotations.description):
                if an == "T0":
                    raw_run.annotations.description[index] = "B"
                if an == "T1":
                    raw_run.annotations.description[index] = "LR"
                if an == "T2":
                    raw_run.annotations.description[index] = "F"
        subject_runs.append(raw_run)

    raw_conc = mne.concatenate_raws(subject_runs)
    mne.datasets.eegbci.standardize(raw_conc)
    montage = mne.channels.make_standard_montage('standard_1005')
    raw_conc.set_montage(montage)

    tmin = 0
    tmax = 4
    event_id = dict(F=2, L=3, LR=4, R=5) if exclude_base else dict(B=1, F=2, L=3, LR=4, R=5)
    events, _ = mne.events_from_annotations(raw_conc, event_id=event_id)

    picks = mne.pick_types(raw_conc.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

    epochs = mne.Epochs(raw_conc, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    y = [epochs[index]._name for index in range(len(epochs))]
    xs = np.array([epoch for epoch in epochs])

    return xs, y, raw_conc.ch_names

def standardize_data(X_train, X_test, channels):
    for j in range(channels):
        scaler = StandardScaler()
        scaler.fit(X_train[:, 0, j, :])
        X_train[:, 0, j, :] = scaler.transform(X_train[:, 0, j, :])
        X_test[:, 0, j, :] = scaler.transform(X_test[:, 0, j, :])
    return X_train, X_test

def to_one_hot(y, by_sub=False):
    new_array = np.array(y)
    total_labels = np.unique(new_array)
    mapping = {label: idx for idx, label in enumerate(total_labels)}
    for x in range(len(new_array)):
        new_array[x] = mapping[new_array[x]]
    return keras.utils.to_categorical(new_array)

def run():
    data_path = "/home/work/HCOH/motors/eeg_motor_files"
    save_path = "/home/work/HCOH/motors/eeg_motor_files/dataset.npz"

    # 데이터셋을 로드하거나 생성
    X_train, y_train, X_test, y_test = load_or_create_dataset(data_path, save_path)

    # 결과 저장 경로 설정
    results_path = os.getcwd() + "/results"
    if not os.path.exists(results_path):
        os.makedirs(results_path)

    # 훈련 하이퍼파라미터 설정
    train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009,
                  'LearnCurves': True, 'model': 'simple_cnn'}

    model = getModel(train_conf.get('model'))
    model.summary()

    # 모델 훈련
    train(model, X_train, y_train, X_test, y_test, train_conf, results_path)

def train(model, X_train, y_train, X_test, y_test, train_conf, results_path):
    model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=train_conf['lr']), metrics=['accuracy'])
    callbacks = [
        ModelCheckpoint(os.path.join(results_path, 'best_model.h5'), monitor='val_accuracy', verbose=1,
                        save_best_only=True, save_weights_only=True, mode='max'),
        EarlyStopping(monitor='val_accuracy', verbose=1, mode='max', patience=train_conf['patience'])
    ]
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=train_conf['epochs'], batch_size=train_conf['batch_size'], callbacks=callbacks, verbose=1)
    # Evaluate the best model
    model.load_weights(os.path.join(results_path, 'best_model.h5'))
    y_pred = model.predict(X_test).argmax(axis=-1)
    labels = y_test.argmax(axis=-1)
    acc = accuracy_score(labels, y_pred)
    kappa = cohen_kappa_score(labels, y_pred)
    print(f"Test Accuracy: {acc}, Kappa Score: {kappa}")

if __name__ == "__main__":
    run()

Loading dataset from disk...
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 64, 640)]      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 64, 32)         184352    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 1, 32, 32)         0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 1, 32, 32)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 1, 32, 64)         18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 1, 16, 64)         0         
 g2D)                         

2024-08-22 12:49:52.200010: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:1021] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-22 12:49:52.375604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-08-22 12:49:53.679483: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4888cba4d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-22 12:49:53.679618: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): CUDA GPU, Compute Capability 8.0
2024-08-22 12:49:53.679640: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): CUDA GPU, Compute Capability 8.0
2024-08-22 12:49:53.685009: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `ML

256/256 [==============================] - ETA: 0s - loss: 1.4095 - accuracy: 0.2973
Epoch 1: val_accuracy improved from -inf to 0.31758, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 5s 8ms/step - loss: 1.4095 - accuracy: 0.2973 - val_loss: 1.3712 - val_accuracy: 0.3176
Epoch 2/500
256/256 [==============================] - ETA: 0s - loss: 1.3721 - accuracy: 0.3208
Epoch 2: val_accuracy improved from 0.31758 to 0.32967, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 6ms/step - loss: 1.3721 - accuracy: 0.3208 - val_loss: 1.3640 - val_accuracy: 0.3297
Epoch 3/500
256/256 [==============================] - ETA: 0s - loss: 1.3591 - accuracy: 0.3485
Epoch 3: val_accuracy improved from 0.32967 to 0.38791, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.3591 - accuracy: 0.3485 - val_loss: 1.3387 - val_accuracy: 0.3879
E

Epoch 28/500
248/256 [============================>.] - ETA: 0s - loss: 0.9129 - accuracy: 0.6215
Epoch 28: val_accuracy did not improve from 0.54396
256/256 [==============================] - 1s 5ms/step - loss: 0.9142 - accuracy: 0.6200 - val_loss: 1.1063 - val_accuracy: 0.5264
Epoch 29/500
254/256 [============================>.] - ETA: 0s - loss: 0.8970 - accuracy: 0.6280
Epoch 29: val_accuracy did not improve from 0.54396
256/256 [==============================] - 1s 5ms/step - loss: 0.8975 - accuracy: 0.6283 - val_loss: 1.1788 - val_accuracy: 0.5198
Epoch 30/500
247/256 [===========================>..] - ETA: 0s - loss: 0.9053 - accuracy: 0.6321
Epoch 30: val_accuracy did not improve from 0.54396
256/256 [==============================] - 1s 5ms/step - loss: 0.9094 - accuracy: 0.6295 - val_loss: 1.1485 - val_accuracy: 0.5209
Epoch 31/500
255/256 [============================>.] - ETA: 0s - loss: 0.9025 - accuracy: 0.6338
Epoch 31: val_accuracy did not improve from 0.54396
256/256

Epoch 57/500
252/256 [============================>.] - ETA: 0s - loss: 0.7143 - accuracy: 0.7080
Epoch 57: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.7148 - accuracy: 0.7080 - val_loss: 1.2138 - val_accuracy: 0.5022
Epoch 58/500
247/256 [===========================>..] - ETA: 0s - loss: 0.7281 - accuracy: 0.7122
Epoch 58: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.7291 - accuracy: 0.7121 - val_loss: 1.2194 - val_accuracy: 0.5165
Epoch 59/500
252/256 [============================>.] - ETA: 0s - loss: 0.7044 - accuracy: 0.7207
Epoch 59: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.7037 - accuracy: 0.7212 - val_loss: 1.2837 - val_accuracy: 0.5209
Epoch 60/500
255/256 [============================>.] - ETA: 0s - loss: 0.7215 - accuracy: 0.7142
Epoch 60: val_accuracy did not improve from 0.54505
256/256

253/256 [============================>.] - ETA: 0s - loss: 0.5301 - accuracy: 0.8013
Epoch 115: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.5304 - accuracy: 0.8011 - val_loss: 1.6589 - val_accuracy: 0.5044
Epoch 116/500
250/256 [============================>.] - ETA: 0s - loss: 0.5154 - accuracy: 0.8018
Epoch 116: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.5152 - accuracy: 0.8015 - val_loss: 1.6635 - val_accuracy: 0.5187
Epoch 117/500
255/256 [============================>.] - ETA: 0s - loss: 0.5005 - accuracy: 0.8113
Epoch 117: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.5006 - accuracy: 0.8116 - val_loss: 1.6551 - val_accuracy: 0.5022
Epoch 118/500
248/256 [============================>.] - ETA: 0s - loss: 0.5113 - accuracy: 0.8085
Epoch 118: val_accuracy did not improve from 0.54505
256/256 [====

Epoch 144/500
255/256 [============================>.] - ETA: 0s - loss: 0.4560 - accuracy: 0.8252
Epoch 144: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.4556 - accuracy: 0.8254 - val_loss: 1.6699 - val_accuracy: 0.5044
Epoch 145/500
245/256 [===========================>..] - ETA: 0s - loss: 0.4508 - accuracy: 0.8337
Epoch 145: val_accuracy did not improve from 0.54505
256/256 [==============================] - 1s 5ms/step - loss: 0.4540 - accuracy: 0.8328 - val_loss: 1.6660 - val_accuracy: 0.4967
Epoch 145: early stopping
29/29 [==============================] - 0s 2ms/step
Test Accuracy: 0.545054945054945, Kappa Score: 0.3933216100794219


In [21]:
import os
import numpy as np
import mne
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.metrics import accuracy_score, cohen_kappa_score
from tensorflow import keras

def load_or_create_dataset(data_path, save_path):
    if os.path.exists(save_path):
        print("Loading dataset from disk...")
        npzfile = np.load(save_path)
        return npzfile['x_train'], npzfile['y_train'], npzfile['x_test'], npzfile['y_test']

    print("Loading dataset from source...")
    exclude = [38, 88, 89, 92, 100, 104]
    subjects = [n for n in np.arange(1, 110) if n not in exclude]

    xs = []
    ys = []
    for subject in subjects:
        x, y, _ = load_subject_data(subject, data_path, True)
        xs.append(x)
        ys.append(y)
    data_x = np.concatenate(xs)
    data_y = np.concatenate(ys)

    N_tr, N_ch, _ = data_x.shape
    data_x = data_x[:, :, :640].reshape(N_tr, 1, N_ch, -1)
    y_one_hot = to_one_hot(data_y, by_sub=False)

    # Create Validation/test
    x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(
        data_x,
        y_one_hot,
        stratify=y_one_hot,
        test_size=0.10,
        random_state=42
    )

    # Scale independently train/test
    x_train_scaled_raw, x_test_valid_scaled_raw = standardize_data(x_train_raw, x_valid_test_raw, N_ch)

    # Save dataset to disk
    np.savez(save_path, x_train=x_train_scaled_raw, y_train=y_train_raw, x_test=x_test_valid_scaled_raw, y_test=y_valid_test_raw)

    print(x_train_scaled_raw.shape, x_test_valid_scaled_raw.shape)
    return x_train_scaled_raw, y_train_raw, x_test_valid_scaled_raw, y_valid_test_raw

def load_subject_data(subject, data_path, exclude_base=False):
    runs = [4, 6, 8, 10, 12, 14]
    task2 = [4, 8, 12]
    task4 = [6, 10, 14]
    sub_name = f"S{subject:03d}"
    sub_folder = os.path.join(data_path, sub_name)
    subject_runs = []

    for run in runs:
        path_run = os.path.join(sub_folder, f"{sub_name}R{run:02d}.edf")
        raw_run = mne.io.read_raw_edf(path_run, preload=True)
        len_run = np.sum(raw_run._annotations.duration)
        if len_run > 124:
            raw_run.crop(tmax=124)

        # Update annotations to match tasks
        if run in task2:
            for i, desc in enumerate(raw_run.annotations.description):
                if desc == "T0":
                    raw_run.annotations.description[i] = "B"
                elif desc == "T1":
                    raw_run.annotations.description[i] = "L"
                elif desc == "T2":
                    raw_run.annotations.description[i] = "R"
        if run in task4:
            for i, desc in enumerate(raw_run.annotations.description):
                if desc == "T0":
                    raw_run.annotations.description[i] = "B"
                elif desc == "T1":
                    raw_run.annotations.description[i] = "LR"
                elif desc == "T2":
                    raw_run.annotations.description[i] = "F"
        subject_runs.append(raw_run)

    raw_conc = mne.concatenate_raws(subject_runs)
    mne.datasets.eegbci.standardize(raw_conc)
    montage = mne.channels.make_standard_montage('standard_1005')
    raw_conc.set_montage(montage)

    tmin, tmax = 0, 4
    event_id = dict(F=2, L=3, LR=4, R=5) if exclude_base else dict(B=1, F=2, L=3, LR=4, R=5)
    events, _ = mne.events_from_annotations(raw_conc, event_id=event_id)

    picks = mne.pick_types(raw_conc.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

    epochs = mne.Epochs(raw_conc, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    y = [epochs[index]._name for index in range(len(epochs))]
    xs = np.array([epoch for epoch in epochs])

    return xs, y, raw_conc.ch_names

def standardize_data(X_train, X_test, channels):
    for j in range(channels):
        scaler = StandardScaler()
        scaler.fit(X_train[:, 0, j, :])
        X_train[:, 0, j, :] = scaler.transform(X_train[:, 0, j, :])
        X_test[:, 0, j, :] = scaler.transform(X_test[:, 0, j, :])
    return X_train, X_test

def to_one_hot(y, by_sub=False):
    new_array = np.array(y)
    total_labels = np.unique(new_array)
    mapping = {label: idx for idx, label in enumerate(total_labels)}
    for x in range(len(new_array)):
        new_array[x] = mapping[new_array[x]]
    return keras.utils.to_categorical(new_array)

def getModel(model_type='simple_cnn'):
    if model_type == 'simple_cnn':
        model = keras.Sequential([
            keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(1, 64, 640)),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 320)
            keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 160)
            keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 80)
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(4, activation='softmax')
        ])
        return model
    return None
def run_kfold(data_x, data_y, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits)
    
    all_accuracies = []
    all_kappas = []
    
    for train_index, test_index in skf.split(data_x, data_y.argmax(axis=1)):
        X_train, X_test = data_x[train_index], data_x[test_index]
        y_train, y_test = data_y[train_index], data_y[test_index]
        
        # 데이터 표준화
        X_train, X_test = standardize_data(X_train, X_test, X_train.shape[2])

        # 모델 초기화 및 학습
        model = getModel('simple_cnn')
        train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009}
        results_path = os.getcwd() + "/results"
        if not os.path.exists(results_path):
            os.makedirs(results_path)
        train(model, X_train, y_train, X_test, y_test, train_conf, results_path)
        
        # 모델 평가
        y_pred = model.predict(X_test).argmax(axis=-1)
        labels = y_test.argmax(axis=-1)
        acc = accuracy_score(labels, y_pred)
        kappa = cohen_kappa_score(labels, y_pred)
        all_accuracies.append(acc)
        all_kappas.append(kappa)
    
    print(f"Average Accuracy: {np.mean(all_accuracies)}, Average Kappa Score: {np.mean(all_kappas)}")

def train(model, X_train, y_train, X_test, y_test, train_conf, results_path):
    model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=train_conf['lr']), metrics=['accuracy'])
    callbacks = [
        ModelCheckpoint(os.path.join(results_path, 'best_model.h5'), monitor='val_accuracy', verbose=1,
                        save_best_only=True, save_weights_only=True, mode='max'),
        EarlyStopping(monitor='val_accuracy', verbose=1, mode='max', patience=train_conf['patience'])
    ]
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=train_conf['epochs'], batch_size=train_conf['batch_size'], callbacks=callbacks, verbose=1)

if __name__ == "__main__":
    data_path = "/home/work/HCOH/motors/eeg_motor_files"
    save_path = "/home/work/HCOH/motors/eeg_motor_files/dataset.npz"
    x_train, y_train, x_test, y_test = load_or_create_dataset(data_path, save_path)
    
    # Combine train and test data for cross-validation
    data_x = np.concatenate([x_train, x_test], axis=0)
    data_y = np.concatenate([y_train, y_test], axis=0)

    # Perform K-fold cross-validation
    run_kfold(data_x, data_y, n_splits=10)

Loading dataset from disk...
Epoch 1/500


2024-08-26 04:12:22.945832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-08-26 04:12:23.453923: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb7e0ccd070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-26 04:12:23.453973: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): CUDA GPU, Compute Capability 8.0
2024-08-26 04:12:23.453981: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): CUDA GPU, Compute Capability 8.0
2024-08-26 04:12:23.462512: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-26 04:12:23.554327: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


256/256 [==============================] - ETA: 0s - loss: 1.3684 - accuracy: 0.3212
Epoch 1: val_accuracy improved from -inf to 0.37253, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 4s 8ms/step - loss: 1.3684 - accuracy: 0.3212 - val_loss: 1.3327 - val_accuracy: 0.3725
Epoch 2/500
256/256 [==============================] - ETA: 0s - loss: 1.2672 - accuracy: 0.4383
Epoch 2: val_accuracy improved from 0.37253 to 0.48681, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.2672 - accuracy: 0.4383 - val_loss: 1.1923 - val_accuracy: 0.4868
Epoch 3/500
256/256 [==============================] - ETA: 0s - loss: 1.1447 - accuracy: 0.5276
Epoch 3: val_accuracy improved from 0.48681 to 0.51429, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1447 - accuracy: 0.5276 - val_loss: 1.1394 - val_accuracy: 0.5143
E

Epoch 29/500
248/256 [============================>.] - ETA: 0s - loss: 0.1040 - accuracy: 0.9671
Epoch 29: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 4ms/step - loss: 0.1042 - accuracy: 0.9671 - val_loss: 3.8407 - val_accuracy: 0.5110
Epoch 30/500
254/256 [============================>.] - ETA: 0s - loss: 0.1238 - accuracy: 0.9585
Epoch 30: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 5ms/step - loss: 0.1237 - accuracy: 0.9586 - val_loss: 3.9496 - val_accuracy: 0.5077
Epoch 31/500
255/256 [============================>.] - ETA: 0s - loss: 0.1322 - accuracy: 0.9577
Epoch 31: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 5ms/step - loss: 0.1324 - accuracy: 0.9576 - val_loss: 3.8045 - val_accuracy: 0.5066
Epoch 32/500
255/256 [============================>.] - ETA: 0s - loss: 0.1625 - accuracy: 0.9487
Epoch 32: val_accuracy did not improve from 0.53626
256/256

247/256 [===========================>..] - ETA: 0s - loss: 0.0889 - accuracy: 0.9729
Epoch 58: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 4ms/step - loss: 0.0892 - accuracy: 0.9725 - val_loss: 4.8316 - val_accuracy: 0.5143
Epoch 59/500
249/256 [============================>.] - ETA: 0s - loss: 0.0878 - accuracy: 0.9743
Epoch 59: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 4ms/step - loss: 0.0871 - accuracy: 0.9745 - val_loss: 4.3555 - val_accuracy: 0.4934
Epoch 60/500
244/256 [===========================>..] - ETA: 0s - loss: 0.0700 - accuracy: 0.9790
Epoch 60: val_accuracy did not improve from 0.53626
256/256 [==============================] - 1s 5ms/step - loss: 0.0689 - accuracy: 0.9790 - val_loss: 5.3361 - val_accuracy: 0.4912
Epoch 61/500
247/256 [===========================>..] - ETA: 0s - loss: 0.0550 - accuracy: 0.9843
Epoch 61: val_accuracy did not improve from 0.53626
256/256 [===========

...

253/256 [============================>.] - ETA: 0s - loss: 0.0029 - accuracy: 0.9995
Epoch 109: val_accuracy did not improve from 0.53575
256/256 [==============================] - 1s 5ms/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 7.0649 - val_accuracy: 0.5182
Epoch 110/500
249/256 [============================>.] - ETA: 0s - loss: 3.1221e-04 - accuracy: 1.0000
Epoch 110: val_accuracy did not improve from 0.53575
256/256 [==============================] - 1s 5ms/step - loss: 3.4483e-04 - accuracy: 1.0000 - val_loss: 7.3222 - val_accuracy: 0.5193
Epoch 111/500
253/256 [============================>.] - ETA: 0s - loss: 1.8934e-04 - accuracy: 1.0000
Epoch 111: val_accuracy did not improve from 0.53575
256/256 [==============================] - 1s 5ms/step - loss: 1.8893e-04 - accuracy: 1.0000 - val_loss: 7.4812 - val_accuracy: 0.5193
Epoch 112/500
252/256 [============================>.] - ETA: 0s - loss: 1.3802e-04 - accuracy: 1.0000
Epoch 112: val_accuracy did not improve from 0

Epoch 3/500
256/256 [==============================] - ETA: 0s - loss: 1.1066 - accuracy: 0.5424
Epoch 3: val_accuracy improved from 0.50165 to 0.51595, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1066 - accuracy: 0.5424 - val_loss: 1.1658 - val_accuracy: 0.5160
Epoch 4/500
254/256 [============================>.] - ETA: 0s - loss: 1.0270 - accuracy: 0.5821
Epoch 4: val_accuracy did not improve from 0.51595
256/256 [==============================] - 1s 5ms/step - loss: 1.0269 - accuracy: 0.5820 - val_loss: 1.1737 - val_accuracy: 0.5061
Epoch 5/500
254/256 [============================>.] - ETA: 0s - loss: 0.9409 - accuracy: 0.6185
Epoch 5: val_accuracy did not improve from 0.51595
256/256 [==============================] - 1s 5ms/step - loss: 0.9421 - accuracy: 0.6177 - val_loss: 1.1870 - val_accuracy: 0.4796
Epoch 6/500
247/256 [===========================>..] - ETA: 0s - loss: 0.8845 - accuracy: 0.6492
Epoch 6

Epoch 32/500
248/256 [============================>.] - ETA: 0s - loss: 0.1203 - accuracy: 0.9626
Epoch 32: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.1244 - accuracy: 0.9607 - val_loss: 3.8296 - val_accuracy: 0.4873
Epoch 33/500
255/256 [============================>.] - ETA: 0s - loss: 0.1501 - accuracy: 0.9498
Epoch 33: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.1501 - accuracy: 0.9497 - val_loss: 4.1134 - val_accuracy: 0.4730
Epoch 34/500
253/256 [============================>.] - ETA: 0s - loss: 0.1418 - accuracy: 0.9534
Epoch 34: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.1421 - accuracy: 0.9536 - val_loss: 4.3458 - val_accuracy: 0.4851
Epoch 35/500
248/256 [============================>.] - ETA: 0s - loss: 0.1087 - accuracy: 0.9643
Epoch 35: val_accuracy did not improve from 0.53465
256/256

252/256 [============================>.] - ETA: 0s - loss: 0.0670 - accuracy: 0.9803
Epoch 61: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0687 - accuracy: 0.9797 - val_loss: 5.0692 - val_accuracy: 0.4972
Epoch 62/500
250/256 [============================>.] - ETA: 0s - loss: 0.1197 - accuracy: 0.9668
Epoch 62: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.1192 - accuracy: 0.9667 - val_loss: 5.0909 - val_accuracy: 0.5105
Epoch 63/500
245/256 [===========================>..] - ETA: 0s - loss: 0.0884 - accuracy: 0.9742
Epoch 63: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 4ms/step - loss: 0.0869 - accuracy: 0.9746 - val_loss: 5.2181 - val_accuracy: 0.4994
Epoch 64/500
248/256 [============================>.] - ETA: 0s - loss: 0.0413 - accuracy: 0.9869
Epoch 64: val_accuracy did not improve from 0.53465
256/256 [===========

246/256 [===========================>..] - ETA: 0s - loss: 0.0309 - accuracy: 0.9919
Epoch 90: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0305 - accuracy: 0.9919 - val_loss: 6.2130 - val_accuracy: 0.4961
Epoch 91/500
249/256 [============================>.] - ETA: 0s - loss: 0.0203 - accuracy: 0.9951
Epoch 91: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0206 - accuracy: 0.9949 - val_loss: 6.6606 - val_accuracy: 0.5127
Epoch 92/500
247/256 [===========================>..] - ETA: 0s - loss: 0.0398 - accuracy: 0.9889
Epoch 92: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0446 - accuracy: 0.9874 - val_loss: 6.3846 - val_accuracy: 0.5127
Epoch 93/500
245/256 [===========================>..] - ETA: 0s - loss: 0.0984 - accuracy: 0.9693
Epoch 93: val_accuracy did not improve from 0.53465
256/256 [===========

In [24]:
import os
import numpy as np
import mne
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.metrics import accuracy_score, cohen_kappa_score
from tensorflow import keras

# 데이터셋을 로드하거나 생성하는 함수
def load_or_create_dataset(data_path, save_path):
    if os.path.exists(save_path):
        print("Loading dataset from disk...")
        npzfile = np.load(save_path)
        return npzfile['x_train'], npzfile['y_train'], npzfile['x_test'], npzfile['y_test']

    print("Loading dataset from source...")
    exclude = [38, 88, 89, 92, 100, 104]
    subjects = [n for n in range(1, 110) if n not in exclude]

    xs, ys = [], []
    for subject in subjects:
        x, y, _ = load_subject_data(subject, data_path, exclude_base=True)
        xs.append(x)
        ys.append(y)
    data_x = np.concatenate(xs)
    data_y = np.concatenate(ys)

    # 데이터셋 전처리
    N_tr, N_ch, _ = data_x.shape
    data_x = data_x[:, :, :640].reshape(N_tr, 1, N_ch, -1)
    y_one_hot = to_one_hot(data_y)

    # 학습 및 검증 세트 생성
    x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(
        data_x, y_one_hot, stratify=y_one_hot, test_size=0.10, random_state=42)

    # 데이터 표준화
    x_train_scaled_raw, x_test_valid_scaled_raw = standardize_data(x_train_raw, x_valid_test_raw, N_ch)

    # 디스크에 데이터셋 저장
    np.savez(save_path, x_train=x_train_scaled_raw, y_train=y_train_raw, x_test=x_test_valid_scaled_raw, y_test=y_valid_test_raw)

    print(x_train_scaled_raw.shape, x_test_valid_scaled_raw.shape)
    return x_train_scaled_raw, y_train_raw, x_test_valid_scaled_raw, y_valid_test_raw

# 피험자 데이터를 로드하는 함수
def load_subject_data(subject, data_path, exclude_base=False):
    runs = [4, 6, 8, 10, 12, 14]
    task2 = [4, 8, 12]
    task4 = [6, 10, 14]
    sub_name = f"S{subject:03d}"
    sub_folder = os.path.join(data_path, sub_name)
    subject_runs = []

    for run in runs:
        path_run = os.path.join(sub_folder, f"{sub_name}R{run:02d}.edf")
        raw_run = mne.io.read_raw_edf(path_run, preload=True)
        if np.sum(raw_run._annotations.duration) > 124:
            raw_run.crop(tmax=124)

        # 작업에 맞게 주석 업데이트
        for i, desc in enumerate(raw_run.annotations.description):
            if run in task2:
                raw_run.annotations.description[i] = {"T0": "B", "T1": "L", "T2": "R"}.get(desc, desc)
            if run in task4:
                raw_run.annotations.description[i] = {"T0": "B", "T1": "LR", "T2": "F"}.get(desc, desc)

        subject_runs.append(raw_run)

    raw_conc = mne.concatenate_raws(subject_runs)
    mne.datasets.eegbci.standardize(raw_conc)
    raw_conc.set_montage(mne.channels.make_standard_montage('standard_1005'))

    tmin, tmax = 0, 4
    event_id = dict(F=2, L=3, LR=4, R=5) if exclude_base else dict(B=1, F=2, L=3, LR=4, R=5)
    events, _ = mne.events_from_annotations(raw_conc, event_id=event_id)

    picks = mne.pick_types(raw_conc.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
    epochs = mne.Epochs(raw_conc, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    return np.array([epoch for epoch in epochs]), [epochs[index]._name for index in range(len(epochs))], raw_conc.ch_names

# 데이터 표준화 함수
def standardize_data(X_train, X_test, channels):
    for j in range(channels):
        scaler = StandardScaler()
        scaler.fit(X_train[:, 0, j, :])
        X_train[:, 0, j, :] = scaler.transform(X_train[:, 0, j, :])
        X_test[:, 0, j, :] = scaler.transform(X_test[:, 0, j, :])
    return X_train, X_test

# 레이블을 원-핫 인코딩으로 변환하는 함수
def to_one_hot(y):
    new_array = np.array(y)
    mapping = {label: idx for idx, label in enumerate(np.unique(new_array))}
    new_array = np.array([mapping[label] for label in new_array])
    return keras.utils.to_categorical(new_array)

def getModel(model_type='simple_cnn'):
    if model_type == 'simple_cnn':
        model = keras.Sequential([
            keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(1, 64, 640)),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 320)
            keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 160)
            keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            keras.layers.MaxPooling2D(pool_size=(1, 2)),  # 크기: (1, 64, 80)
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(4, activation='softmax')
        ])
        return model
    else:
        raise ValueError("Invalid model type provided.")

# 교차 검증을 수행하는 함수
def run_kfold(data_x, data_y, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits)
    all_accuracies, all_kappas = [], []
    
    for train_index, test_index in skf.split(data_x, data_y.argmax(axis=1)):
        X_train, X_test = data_x[train_index], data_x[test_index]
        y_train, y_test = data_y[train_index], data_y[test_index]

        # 데이터 표준화
        X_train, X_test = standardize_data(X_train, X_test, X_train.shape[2])

        # 모델 학습 및 평가
        model = getModel('simple_cnn')
        train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 10, 'lr': 0.0009}
        results_path = os.path.join(os.getcwd(), "results")
        os.makedirs(results_path, exist_ok=True)
        train(model, X_train, y_train, X_test, y_test, train_conf, results_path)

        y_pred = model.predict(X_test).argmax(axis=-1)
        labels = y_test.argmax(axis=-1)
        all_accuracies.append(accuracy_score(labels, y_pred))
        all_kappas.append(cohen_kappa_score(labels, y_pred))
    
    print(f"Average Accuracy: {np.mean(all_accuracies)}, Average Kappa Score: {np.mean(all_kappas)}")

# 모델을 학습하는 함수
def train(model, X_train, y_train, X_test, y_test, train_conf, results_path):
    model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=train_conf['lr']), metrics=['accuracy'])
    callbacks = [
        ModelCheckpoint(os.path.join(results_path, 'best_model.h5'), monitor='val_accuracy', verbose=1,
                        save_best_only=True, save_weights_only=True, mode='max'),
        EarlyStopping(monitor='val_accuracy', verbose=1, mode='max', patience=train_conf['patience'], restore_best_weights=True)
    ]
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=train_conf['epochs'], batch_size=train_conf['batch_size'], callbacks=callbacks, verbose=1)

if __name__ == "__main__":
    data_path = "/home/work/HCOH/motors/eeg_motor_files"
    save_path = "/home/work/HCOH/motors/eeg_motor_files/dataset.npz"
    
    # 데이터셋 로드 또는 생성
    x_train, y_train, x_test, y_test = load_or_create_dataset(data_path, save_path)
    
    # 학습 데이터와 테스트 데이터를 합쳐서 K-Fold 교차 검증 수행
    data_x = np.concatenate([x_train, x_test], axis=0)
    data_y = np.concatenate([y_train, y_test], axis=0)

    # K-Fold 교차 검증 실행
    run_kfold(data_x, data_y, n_splits=10)

    # 모델 테스트
    model = getModel('simple_cnn')
    train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009}
    results_path = os.getcwd() + "/results"

    train(model, x_train, y_train, x_test, y_test, train_conf, results_path)

    # 최종 테스트 결과 출력
    y_pred = model.predict(x_test).argmax(axis=-1)
    labels = y_test.argmax(axis=-1)
    acc = accuracy_score(labels, y_pred)
    kappa = cohen_kappa_score(labels, y_pred)

    print(f"Final Test Accuracy: {acc}, Kappa Score: {kappa}")

Loading dataset from disk...
Epoch 1/500
254/256 [============================>.] - ETA: 0s - loss: 1.3742 - accuracy: 0.3148
Epoch 1: val_accuracy improved from -inf to 0.38571, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 3s 6ms/step - loss: 1.3738 - accuracy: 0.3164 - val_loss: 1.3425 - val_accuracy: 0.3857
Epoch 2/500
245/256 [===========================>..] - ETA: 0s - loss: 1.2799 - accuracy: 0.4295
Epoch 2: val_accuracy improved from 0.38571 to 0.45604, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.2816 - accuracy: 0.4317 - val_loss: 1.2424 - val_accuracy: 0.4560
Epoch 3/500
248/256 [============================>.] - ETA: 0s - loss: 1.1539 - accuracy: 0.5174
Epoch 3: val_accuracy improved from 0.45604 to 0.51099, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1521 - accuracy: 0.5182 - 

Epoch 13/500
246/256 [===========================>..] - ETA: 0s - loss: 0.4445 - accuracy: 0.8354
Epoch 13: val_accuracy did not improve from 0.56923
256/256 [==============================] - 1s 5ms/step - loss: 0.4467 - accuracy: 0.8351 - val_loss: 1.7407 - val_accuracy: 0.5297
Epoch 14/500
255/256 [============================>.] - ETA: 0s - loss: 0.3996 - accuracy: 0.8507
Epoch 14: val_accuracy did not improve from 0.56923
256/256 [==============================] - 1s 5ms/step - loss: 0.3996 - accuracy: 0.8507 - val_loss: 1.8049 - val_accuracy: 0.5440
Epoch 15/500
245/256 [===========================>..] - ETA: 0s - loss: 0.3627 - accuracy: 0.8704
Epoch 15: val_accuracy did not improve from 0.56923
256/256 [==============================] - 1s 4ms/step - loss: 0.3663 - accuracy: 0.8694 - val_loss: 1.8289 - val_accuracy: 0.5440
Epoch 16/500
249/256 [============================>.] - ETA: 0s - loss: 0.3307 - accuracy: 0.8795
Epoch 16: val_accuracy did not improve from 0.56923
Restori

Epoch 6/500
247/256 [===========================>..] - ETA: 0s - loss: 0.9032 - accuracy: 0.6399
Epoch 6: val_accuracy did not improve from 0.54945
256/256 [==============================] - 1s 4ms/step - loss: 0.9028 - accuracy: 0.6401 - val_loss: 1.1134 - val_accuracy: 0.5275
Epoch 7/500
250/256 [============================>.] - ETA: 0s - loss: 0.8364 - accuracy: 0.6645
Epoch 7: val_accuracy did not improve from 0.54945
256/256 [==============================] - 1s 5ms/step - loss: 0.8359 - accuracy: 0.6656 - val_loss: 1.2047 - val_accuracy: 0.5396
Epoch 8/500
247/256 [===========================>..] - ETA: 0s - loss: 0.7652 - accuracy: 0.6991
Epoch 8: val_accuracy did not improve from 0.54945
256/256 [==============================] - 1s 5ms/step - loss: 0.7642 - accuracy: 0.7000 - val_loss: 1.2172 - val_accuracy: 0.5451
Epoch 9/500
256/256 [==============================] - ETA: 0s - loss: 0.7099 - accuracy: 0.7221
Epoch 9: val_accuracy did not improve from 0.54945
256/256 [======

Epoch 3/500
253/256 [============================>.] - ETA: 0s - loss: 1.1151 - accuracy: 0.5468
Epoch 3: val_accuracy improved from 0.50989 to 0.54945, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1157 - accuracy: 0.5468 - val_loss: 1.1153 - val_accuracy: 0.5495
Epoch 4/500
251/256 [============================>.] - ETA: 0s - loss: 1.0223 - accuracy: 0.5862
Epoch 4: val_accuracy did not improve from 0.54945
256/256 [==============================] - 1s 5ms/step - loss: 1.0221 - accuracy: 0.5860 - val_loss: 1.1224 - val_accuracy: 0.5165
Epoch 5/500
245/256 [===========================>..] - ETA: 0s - loss: 0.9564 - accuracy: 0.6088
Epoch 5: val_accuracy did not improve from 0.54945
256/256 [==============================] - 1s 5ms/step - loss: 0.9570 - accuracy: 0.6107 - val_loss: 1.1139 - val_accuracy: 0.5385
Epoch 6/500
246/256 [===========================>..] - ETA: 0s - loss: 0.8778 - accuracy: 0.6476
Epoch 6

256/256 [==============================] - ETA: 0s - loss: 1.3655 - accuracy: 0.3267
Epoch 1: val_accuracy improved from -inf to 0.37844, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 3s 6ms/step - loss: 1.3655 - accuracy: 0.3267 - val_loss: 1.3382 - val_accuracy: 0.3784
Epoch 2/500
250/256 [============================>.] - ETA: 0s - loss: 1.2650 - accuracy: 0.4356
Epoch 2: val_accuracy improved from 0.37844 to 0.50825, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.2630 - accuracy: 0.4379 - val_loss: 1.1986 - val_accuracy: 0.5083
Epoch 3/500
249/256 [============================>.] - ETA: 0s - loss: 1.1126 - accuracy: 0.5474
Epoch 3: val_accuracy improved from 0.50825 to 0.53685, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1145 - accuracy: 0.5455 - val_loss: 1.1674 - val_accuracy: 0.5369
E

Epoch 14/500
249/256 [============================>.] - ETA: 0s - loss: 0.3878 - accuracy: 0.8593
Epoch 14: val_accuracy did not improve from 0.56106
256/256 [==============================] - 1s 5ms/step - loss: 0.3893 - accuracy: 0.8586 - val_loss: 1.9803 - val_accuracy: 0.5083
Epoch 15/500
245/256 [===========================>..] - ETA: 0s - loss: 0.3594 - accuracy: 0.8704
Epoch 15: val_accuracy did not improve from 0.56106
Restoring model weights from the end of the best epoch: 5.
256/256 [==============================] - 1s 5ms/step - loss: 0.3601 - accuracy: 0.8695 - val_loss: 2.0743 - val_accuracy: 0.5369
Epoch 15: early stopping
29/29 [==============================] - 0s 2ms/step
Epoch 1/500
254/256 [============================>.] - ETA: 0s - loss: 1.3554 - accuracy: 0.3461
Epoch 1: val_accuracy improved from -inf to 0.44004, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 3s 6ms/step - loss: 1.3555 - accuracy: 0.3463 - val_lo

Epoch 8/500
249/256 [============================>.] - ETA: 0s - loss: 0.7602 - accuracy: 0.7031
Epoch 8: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.7623 - accuracy: 0.7017 - val_loss: 1.1864 - val_accuracy: 0.5165
Epoch 9/500
245/256 [===========================>..] - ETA: 0s - loss: 0.6813 - accuracy: 0.7344
Epoch 9: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.6813 - accuracy: 0.7328 - val_loss: 1.2860 - val_accuracy: 0.5099
Epoch 10/500
248/256 [============================>.] - ETA: 0s - loss: 0.6271 - accuracy: 0.7589
Epoch 10: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 4ms/step - loss: 0.6280 - accuracy: 0.7591 - val_loss: 1.2935 - val_accuracy: 0.5297
Epoch 11/500
248/256 [============================>.] - ETA: 0s - loss: 0.5627 - accuracy: 0.7855
Epoch 11: val_accuracy did not improve from 0.54286
256/256 [==

254/256 [============================>.] - ETA: 0s - loss: 0.1247 - accuracy: 0.9612
Epoch 37: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.1247 - accuracy: 0.9612 - val_loss: 4.3111 - val_accuracy: 0.4989
Epoch 38/500
250/256 [============================>.] - ETA: 0s - loss: 0.1048 - accuracy: 0.9672
Epoch 38: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.1042 - accuracy: 0.9674 - val_loss: 4.1955 - val_accuracy: 0.5154
Epoch 39/500
252/256 [============================>.] - ETA: 0s - loss: 0.1172 - accuracy: 0.9652
Epoch 39: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.1164 - accuracy: 0.9654 - val_loss: 4.4364 - val_accuracy: 0.4758
Epoch 40/500
246/256 [===========================>..] - ETA: 0s - loss: 0.0626 - accuracy: 0.9816
Epoch 40: val_accuracy did not improve from 0.54286
256/256 [===========

254/256 [============================>.] - ETA: 0s - loss: 0.1082 - accuracy: 0.9690
Epoch 66: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.1082 - accuracy: 0.9691 - val_loss: 5.1131 - val_accuracy: 0.4967
Epoch 67/500
253/256 [============================>.] - ETA: 0s - loss: 0.0894 - accuracy: 0.9721
Epoch 67: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.0893 - accuracy: 0.9720 - val_loss: 5.3834 - val_accuracy: 0.4901
Epoch 68/500
254/256 [============================>.] - ETA: 0s - loss: 0.0666 - accuracy: 0.9817
Epoch 68: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.0674 - accuracy: 0.9814 - val_loss: 4.7811 - val_accuracy: 0.5022
Epoch 69/500
250/256 [============================>.] - ETA: 0s - loss: 0.0564 - accuracy: 0.9825
Epoch 69: val_accuracy did not improve from 0.54286
256/256 [===========

248/256 [============================>.] - ETA: 0s - loss: 0.0608 - accuracy: 0.9836
Epoch 95: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.0600 - accuracy: 0.9835 - val_loss: 5.9612 - val_accuracy: 0.5022
Epoch 96/500
246/256 [===========================>..] - ETA: 0s - loss: 0.0154 - accuracy: 0.9954
Epoch 96: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.0156 - accuracy: 0.9952 - val_loss: 6.2205 - val_accuracy: 0.5077
Epoch 97/500
249/256 [============================>.] - ETA: 0s - loss: 0.0128 - accuracy: 0.9969
Epoch 97: val_accuracy did not improve from 0.54286
256/256 [==============================] - 1s 5ms/step - loss: 0.0129 - accuracy: 0.9968 - val_loss: 6.2997 - val_accuracy: 0.4945
Epoch 98/500
252/256 [============================>.] - ETA: 0s - loss: 0.0149 - accuracy: 0.9963
Epoch 98: val_accuracy did not improve from 0.54286
256/256 [===========

In [27]:
import os
import numpy as np
import mne
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow import keras

# 기존 load_or_create_dataset, load_subject_data, standardize_data, to_one_hot, getModel 함수 생략...

def run_kfold(data_x, data_y, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits)
    
    all_accuracies = []
    all_kappas = []
    all_f1_scores = []
    
    for train_index, test_index in skf.split(data_x, data_y.argmax(axis=1)):
        X_train, X_test = data_x[train_index], data_x[test_index]
        y_train, y_test = data_y[train_index], data_y[test_index]
        
        # 데이터 표준화
        X_train, X_test = standardize_data(X_train, X_test, X_train.shape[2])

        # 모델 초기화 및 학습
        model = getModel('simple_cnn')
        train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009}
        results_path = os.getcwd() + "/results"
        if not os.path.exists(results_path):
            os.makedirs(results_path)
        train(model, X_train, y_train, X_test, y_test, train_conf, results_path)
        
        # 모델 평가
        y_pred = model.predict(X_test).argmax(axis=-1)
        labels = y_test.argmax(axis=-1)
        acc = accuracy_score(labels, y_pred)
        kappa = cohen_kappa_score(labels, y_pred)
        f1 = f1_score(labels, y_pred, average='weighted')  # F1-score 계산
        
        all_accuracies.append(acc)
        all_kappas.append(kappa)
        all_f1_scores.append(f1)
    
    print(f"Average Accuracy: {np.mean(all_accuracies)}, Average Kappa Score: {np.mean(all_kappas)}, Average F1-Score: {np.mean(all_f1_scores)}")

def train(model, X_train, y_train, X_test, y_test, train_conf, results_path):
    model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=train_conf['lr']), metrics=['accuracy'])
    callbacks = [
        ModelCheckpoint(os.path.join(results_path, 'best_model.h5'), monitor='val_accuracy', verbose=1,
                        save_best_only=True, save_weights_only=True, mode='max'),
        EarlyStopping(monitor='val_accuracy', verbose=1, mode='max', patience=train_conf['patience'], restore_best_weights=True)
    ]
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=train_conf['epochs'], batch_size=train_conf['batch_size'], callbacks=callbacks, verbose=1)

# 메인 실행 코드
if __name__ == "__main__":
    data_path = "/home/work/HCOH/motors/eeg_motor_files"
    save_path = "/home/work/HCOH/motors/eeg_motor_files/dataset.npz"
    
    # 데이터셋 로드 또는 생성
    x_train, y_train, x_test, y_test = load_or_create_dataset(data_path, save_path)
    
    # 학습 데이터와 테스트 데이터를 합쳐서 K-Fold 교차 검증 수행
    data_x = np.concatenate([x_train, x_test], axis=0)
    data_y = np.concatenate([y_train, y_test], axis=0)

    # K-Fold 교차 검증 실행
    run_kfold(data_x, data_y, n_splits=10)

    # 모델 테스트
    model = getModel('simple_cnn')
    train_conf = {'batch_size': 32, 'epochs': 500, 'patience': 100, 'lr': 0.0009}
    results_path = os.getcwd() + "/results"

    train(model, x_train, y_train, x_test, y_test, train_conf, results_path)

    # 최종 테스트 결과 출력 및 평가
    y_pred = model.predict(x_test).argmax(axis=-1)
    labels = y_test.argmax(axis=-1)
    acc = accuracy_score(labels, y_pred)
    kappa = cohen_kappa_score(labels, y_pred)
    f1 = f1_score(labels, y_pred, average='weighted')  # 최종 F1-score 계산

    print(f"Final Test Accuracy: {acc}, Kappa Score: {kappa}, F1-Score: {f1}")

    # Confusion Matrix 출력
    cm = confusion_matrix(labels, y_pred)
    print("Confusion Matrix:")
    print(cm)

Loading dataset from disk...
Epoch 1/500
253/256 [============================>.] - ETA: 0s - loss: 1.3724 - accuracy: 0.3194
Epoch 1: val_accuracy improved from -inf to 0.32857, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 3s 6ms/step - loss: 1.3721 - accuracy: 0.3202 - val_loss: 1.3483 - val_accuracy: 0.3286
Epoch 2/500
249/256 [============================>.] - ETA: 0s - loss: 1.2957 - accuracy: 0.4239
Epoch 2: val_accuracy improved from 0.32857 to 0.48791, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.2925 - accuracy: 0.4272 - val_loss: 1.1920 - val_accuracy: 0.4879
Epoch 3/500
245/256 [===========================>..] - ETA: 0s - loss: 1.1348 - accuracy: 0.5309
Epoch 3: val_accuracy improved from 0.48791 to 0.51538, saving model to /home/work/HCOH/results/best_model.h5
256/256 [==============================] - 1s 5ms/step - loss: 1.1349 - accuracy: 0.5319 - 

Epoch 29/500
251/256 [============================>.] - ETA: 0s - loss: 0.2368 - accuracy: 0.9249
Epoch 29: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.2356 - accuracy: 0.9251 - val_loss: 3.2059 - val_accuracy: 0.5143
Epoch 30/500
250/256 [============================>.] - ETA: 0s - loss: 0.1403 - accuracy: 0.9488
Epoch 30: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.1411 - accuracy: 0.9489 - val_loss: 4.0478 - val_accuracy: 0.5121
Epoch 31/500
249/256 [============================>.] - ETA: 0s - loss: 0.0779 - accuracy: 0.9763
Epoch 31: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.0802 - accuracy: 0.9761 - val_loss: 4.1261 - val_accuracy: 0.5121
Epoch 32/500
252/256 [============================>.] - ETA: 0s - loss: 0.0853 - accuracy: 0.9728
Epoch 32: val_accuracy did not improve from 0.54725
256/256

256/256 [==============================] - ETA: 0s - loss: 0.0608 - accuracy: 0.9812
Epoch 58: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.0608 - accuracy: 0.9812 - val_loss: 5.4156 - val_accuracy: 0.5121
Epoch 59/500
253/256 [============================>.] - ETA: 0s - loss: 0.0476 - accuracy: 0.9858
Epoch 59: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.0474 - accuracy: 0.9858 - val_loss: 5.3230 - val_accuracy: 0.5286
Epoch 60/500
251/256 [============================>.] - ETA: 0s - loss: 0.0356 - accuracy: 0.9900
Epoch 60: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 0.0365 - accuracy: 0.9899 - val_loss: 6.1645 - val_accuracy: 0.5088
Epoch 61/500
251/256 [============================>.] - ETA: 0s - loss: 0.0791 - accuracy: 0.9776
Epoch 61: val_accuracy did not improve from 0.54725
256/256 [===========

Epoch 87/500
248/256 [============================>.] - ETA: 0s - loss: 3.2269e-05 - accuracy: 1.0000
Epoch 87: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 3.2066e-05 - accuracy: 1.0000 - val_loss: 8.2730 - val_accuracy: 0.5308
Epoch 88/500
245/256 [===========================>..] - ETA: 0s - loss: 2.8099e-05 - accuracy: 1.0000
Epoch 88: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 2.8194e-05 - accuracy: 1.0000 - val_loss: 8.3545 - val_accuracy: 0.5319
Epoch 89/500
253/256 [============================>.] - ETA: 0s - loss: 2.4594e-05 - accuracy: 1.0000
Epoch 89: val_accuracy did not improve from 0.54725
256/256 [==============================] - 1s 5ms/step - loss: 2.4535e-05 - accuracy: 1.0000 - val_loss: 8.4275 - val_accuracy: 0.5319
Epoch 90/500
245/256 [===========================>..] - ETA: 0s - loss: 2.1453e-05 - accuracy: 1.0000
Epoch 90: val_accuracy did not 

244/256 [===========================>..] - ETA: 0s - loss: 0.8106 - accuracy: 0.6789
Epoch 7: val_accuracy did not improve from 0.55604
256/256 [==============================] - 1s 5ms/step - loss: 0.8140 - accuracy: 0.6776 - val_loss: 1.1561 - val_accuracy: 0.5407
Epoch 8/500
256/256 [==============================] - ETA: 0s - loss: 0.7418 - accuracy: 0.7134
Epoch 8: val_accuracy did not improve from 0.55604
256/256 [==============================] - 1s 5ms/step - loss: 0.7418 - accuracy: 0.7134 - val_loss: 1.2514 - val_accuracy: 0.5516
Epoch 9/500
248/256 [============================>.] - ETA: 0s - loss: 0.6667 - accuracy: 0.7432
Epoch 9: val_accuracy did not improve from 0.55604
256/256 [==============================] - 1s 5ms/step - loss: 0.6682 - accuracy: 0.7430 - val_loss: 1.2558 - val_accuracy: 0.5308
Epoch 10/500
254/256 [============================>.] - ETA: 0s - loss: 0.5961 - accuracy: 0.7736
Epoch 10: val_accuracy did not improve from 0.55604
256/256 [================

...

249/256 [============================>.] - ETA: 0s - loss: 0.0369 - accuracy: 0.9905
Epoch 83: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0366 - accuracy: 0.9904 - val_loss: 6.3468 - val_accuracy: 0.5138
Epoch 84/500
251/256 [============================>.] - ETA: 0s - loss: 0.0961 - accuracy: 0.9739
Epoch 84: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0951 - accuracy: 0.9741 - val_loss: 5.2567 - val_accuracy: 0.4972
Epoch 85/500
250/256 [============================>.] - ETA: 0s - loss: 0.0486 - accuracy: 0.9852
Epoch 85: val_accuracy did not improve from 0.53465
256/256 [==============================] - 1s 5ms/step - loss: 0.0485 - accuracy: 0.9853 - val_loss: 6.0303 - val_accuracy: 0.5083
Epoch 86/500
251/256 [============================>.] - ETA: 0s - loss: 0.0343 - accuracy: 0.9898
Epoch 86: val_accuracy did not improve from 0.53465
256/256 [===========

254/256 [============================>.] - ETA: 0s - loss: 0.8735 - accuracy: 0.6529
Epoch 6: val_accuracy did not improve from 0.53077
256/256 [==============================] - 1s 5ms/step - loss: 0.8742 - accuracy: 0.6526 - val_loss: 1.1776 - val_accuracy: 0.5198
Epoch 7/500
256/256 [==============================] - ETA: 0s - loss: 0.8098 - accuracy: 0.6869
Epoch 7: val_accuracy did not improve from 0.53077
256/256 [==============================] - 1s 5ms/step - loss: 0.8098 - accuracy: 0.6869 - val_loss: 1.2599 - val_accuracy: 0.5286
Epoch 8/500
254/256 [============================>.] - ETA: 0s - loss: 0.7329 - accuracy: 0.7101
Epoch 8: val_accuracy did not improve from 0.53077
256/256 [==============================] - 1s 5ms/step - loss: 0.7338 - accuracy: 0.7097 - val_loss: 1.2567 - val_accuracy: 0.5099
Epoch 9/500
246/256 [===========================>..] - ETA: 0s - loss: 0.6706 - accuracy: 0.7417
Epoch 9: val_accuracy improved from 0.53077 to 0.53187, saving model to /home/

255/256 [============================>.] - ETA: 0s - loss: 0.1359 - accuracy: 0.9576
Epoch 35: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.1356 - accuracy: 0.9577 - val_loss: 3.9512 - val_accuracy: 0.4956
Epoch 36/500
253/256 [============================>.] - ETA: 0s - loss: 0.1352 - accuracy: 0.9587
Epoch 36: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.1343 - accuracy: 0.9590 - val_loss: 4.0250 - val_accuracy: 0.5055
Epoch 37/500
247/256 [===========================>..] - ETA: 0s - loss: 0.1278 - accuracy: 0.9579
Epoch 37: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 4ms/step - loss: 0.1288 - accuracy: 0.9574 - val_loss: 3.7842 - val_accuracy: 0.5066
Epoch 38/500
255/256 [============================>.] - ETA: 0s - loss: 0.0637 - accuracy: 0.9814
Epoch 38: val_accuracy did not improve from 0.53187
256/256 [===========

256/256 [==============================] - ETA: 0s - loss: 0.0758 - accuracy: 0.9783
Epoch 64: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.0758 - accuracy: 0.9783 - val_loss: 4.8925 - val_accuracy: 0.5000
Epoch 65/500
248/256 [============================>.] - ETA: 0s - loss: 0.0316 - accuracy: 0.9893
Epoch 65: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 4ms/step - loss: 0.0312 - accuracy: 0.9896 - val_loss: 5.2170 - val_accuracy: 0.5198
Epoch 66/500
256/256 [==============================] - ETA: 0s - loss: 0.0207 - accuracy: 0.9939
Epoch 66: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.0207 - accuracy: 0.9939 - val_loss: 5.6975 - val_accuracy: 0.5242
Epoch 67/500
249/256 [============================>.] - ETA: 0s - loss: 0.0246 - accuracy: 0.9917
Epoch 67: val_accuracy did not improve from 0.53187
256/256 [===========

248/256 [============================>.] - ETA: 0s - loss: 0.0458 - accuracy: 0.9870
Epoch 93: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.0459 - accuracy: 0.9871 - val_loss: 6.2982 - val_accuracy: 0.5088
Epoch 94/500
251/256 [============================>.] - ETA: 0s - loss: 0.0420 - accuracy: 0.9874
Epoch 94: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.0441 - accuracy: 0.9872 - val_loss: 5.7170 - val_accuracy: 0.4989
Epoch 95/500
245/256 [===========================>..] - ETA: 0s - loss: 0.0334 - accuracy: 0.9913
Epoch 95: val_accuracy did not improve from 0.53187
256/256 [==============================] - 1s 5ms/step - loss: 0.0329 - accuracy: 0.9913 - val_loss: 7.1280 - val_accuracy: 0.5000
Epoch 96/500
246/256 [===========================>..] - ETA: 0s - loss: 0.0161 - accuracy: 0.9956
Epoch 96: val_accuracy did not improve from 0.53187
256/256 [===========